<a href="https://colab.research.google.com/github/AmirGhnbr/Titanic/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np

In [2]:
train_load = pd.read_csv("./drive/MyDrive/train.csv").set_index("PassengerId")
test_load = pd.read_csv("./drive/MyDrive/test.csv").set_index("PassengerId")
len(train_load)

891

In [3]:
X_train,y_train= train_load.drop("Survived",axis=1), train_load["Survived"]

In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 76.6+ KB


In [5]:
X_train.isnull().sum()

Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
X_train.describe()

,Pclass,Age,SibSp,Parch,Fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,20.125000,0.000000,0.000000,7.910400
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,38.000000,1.000000,0.000000,31.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200


## Make pipeline and normalize the data

#### Create full pipeline for numeric data & categoric

In [7]:
## num pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_data = X_train.select_dtypes(include="number")
num_attri = list(num_data)

num_pipeline = Pipeline([
    ("imputer",SimpleImputer(strategy="median")),
    ("Scaler", StandardScaler())
])


In [8]:
## catagorical
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
    ("imputer",SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(sparse=False, handle_unknown="ignore"))
])

cat_data = X_train.select_dtypes(include="object").columns
cat_attri= list(cat_data)
cat_attri

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [9]:
## FULL PIPELINE
from sklearn.compose import ColumnTransformer


full_pipeline_train = ColumnTransformer([
    ("num", num_pipeline, num_attri),
    ("cat", cat_pipeline, cat_attri)
])
transformed_data = full_pipeline_train.fit_transform(X_train[num_attri + cat_attri])
transformed_data

array([[ 0.82737724, -0.56573646,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693,  0.66386103,  0.43279337, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724, -0.25833709, -0.4745452 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693, -0.25833709, -0.4745452 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724,  0.20276197, -0.4745452 , ...,  0.        ,
         1.        ,  0.        ]])

In [10]:
# df_prepared = pd.DataFrame(transformed_data)
# df_prepared.head()

## Test models

In [11]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(transformed_data,y_train)

RandomForestClassifier(random_state=42)

In [12]:
test = full_pipeline_train.transform(test_load[num_attri + cat_attri])
y_label = forest_clf.predict(test)
y_label

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## SVM Classifier

In [13]:
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto")
svm_clf.fit(transformed_data, y_train)

SVC(gamma='auto')

In [14]:
X_test = full_pipeline_train.transform(test_load)
y_test_pred = svm_clf.predict(X_test)
y_test_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
from sklearn.model_selection import cross_val_score

svm_scores = cross_val_score(svm_clf, transformed_data, y_train, cv=10)
svm_scores.mean()

0.6307740324594258

In [16]:
## compare with forest_clf
forest_score = cross_val_score(forest_clf, transformed_data, y_train,cv=10)
forest_score.mean()

0.8361423220973784

In [17]:
## use K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

neighbor_clf = KNeighborsClassifier()
neighbor_clf.fit(transformed_data, y_train)

KNeighborsClassifier()

In [18]:
neighbor_score = cross_val_score(neighbor_clf, transformed_data, y_train,cv=10)
neighbor_score.mean()

0.8036079900124843

In [19]:
y_pred = neighbor_clf.predict(X_test)

## Create function for evaluating couple of models

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

models = {"Logistic Regression": LogisticRegression(),
          "KNN": KNeighborsClassifier(),
          "Extra tree classifier":ExtraTreesClassifier(n_estimators=100),
          "Random Forest": RandomForestClassifier(),
          "Gaussian Process Classifier": GaussianProcessClassifier(kernel=1.0 * RBF(1.0)),
          "multilayer perceptron":MLPClassifier(max_iter=300),
          "ada boost": AdaBoostClassifier(n_estimators=100),
          "decision tree": DecisionTreeClassifier(),
          "xgboost classifier": XGBClassifier()}

In [21]:
def fit_and_score(models, X_train, y_train):
    """
    Fits and evaluates given machine learning models.
    models : a dict of differetn Scikit-Learn machine learning models
    X_train : training data (no labels)
    X_test : testing data (no labels)
    y_train : training labels
    y_test : test labels
    """
    # Set random seed
    np.random.seed(42)
    # Make a dictionary to keep model scores
    model_scores = {}
    # Loop through models
    for name, model in models.items():
        # Fit the model to the data
        model.fit(X_train, y_train)
        model_score = cross_val_score(model, X_train, y_train,cv=10)
        # Evaluate the model and append its score to model_scores
        model_scores[name] = model_score.mean()
    return model_scores

In [22]:
model_scores = fit_and_score(models=models,
                             X_train=transformed_data,
                             y_train=y_train)
model_scores

{'Logistic Regression': 0.8080774032459427,
 'KNN': 0.8036079900124843,
 'Extra tree classifier': 0.8294007490636706,
 'Random Forest': 0.8327340823970036,
 'Gaussian Process Classifier': 0.8271535580524345,
 'multilayer perceptron': 0.8237952559300874,
 'ada boost': 0.8159176029962545,
 'decision tree': 0.8227590511860174,
 'xgboost classifier': 0.8271660424469414}

In [23]:
#### Sort the dictionary
sorted_model_scores = sorted(model_scores.items(), key=lambda x: x[1], reverse=True)
sorted_model_scores

[('Random Forest', 0.8327340823970036),
 ('Extra tree classifier', 0.8294007490636706),
 ('xgboost classifier', 0.8271660424469414),
 ('Gaussian Process Classifier', 0.8271535580524345),
 ('multilayer perceptron', 0.8237952559300874),
 ('decision tree', 0.8227590511860174),
 ('ada boost', 0.8159176029962545),
 ('Logistic Regression', 0.8080774032459427),
 ('KNN', 0.8036079900124843)]

## lets Tune top 3 classifiers : RFC & ETC & XGBC

In [26]:
## tune RFC with gridsearch
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# rfc=RandomForestClassifier()
grid_search_rfc = GridSearchCV(estimator = models["Random Forest"], param_grid = param_grid,
                           cv = 3, n_jobs = -1, verbose = 2)
grid_search_rfc.fit(transformed_data, y_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110],
                         'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100, 200, 300, 1000]},
             verbose=2)

In [31]:
grid_search_rfc.best_params_ , grid_search_rfc.best_estimator_

({'bootstrap': True,
  'max_depth': 80,
  'max_features': 2,
  'min_samples_leaf': 3,
  'min_samples_split': 8,
  'n_estimators': 100},
 RandomForestClassifier(max_depth=80, max_features=2, min_samples_leaf=3,
                        min_samples_split=8))

In [32]:
rfc_score = cross_val_score(grid_search_rfc.best_estimator_, transformed_data, y_train, cv=3,scoring="accuracy")
rfc_score.mean()

0.6161616161616161

##### Default hyperparameters for rfc was the best hyperparameters :|

In [33]:
## Tune Extra tree Clasiifier
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


grid_search_etc = GridSearchCV(estimator = models["Extra tree classifier"], param_grid = param_grid,
                           cv = 3, n_jobs = -1, verbose = 2)
grid_search_etc.fit(transformed_data, y_train)


Fitting 3 folds for each of 288 candidates, totalling 864 fits


GridSearchCV(cv=3, estimator=ExtraTreesClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110],
                         'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100, 200, 300, 1000]},
             verbose=2)

In [34]:
etc_score = cross_val_score(grid_search_etc.best_estimator_, transformed_data, y_train, cv=3,scoring="accuracy")
etc_score.mean()

0.6161616161616161